In [1]:
import pandas as pd
import joblib
import optuna
from keras import regularizers
from tensorflow.keras import callbacks
import tensorflow as tf
from sklearn.model_selection import KFold
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
import os
from optuna.samplers import TPESampler
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
print(os.getcwd())

C:\Users\Joshua\Desktop\nba_third_try\model


In [2]:
df=pd.read_excel('../cleaning/final_df.xlsx')
# df=df.drop('Unnamed: 0')
df.head(2)

,Unnamed: 0,Visitor,Decimal_Visitor,Home,Home_Decimal,Season,Home_Winner,Offense_MP_Visitor,Offense_FG_Visitor,Offense_FGA_Visitor,...,Defense_FTA_Home,Defense_FT%_Home,Defense_ORB_Home,Defense_DRB_Home,Defense_TRB_Home,Defense_AST_Home,Defense_STL_Home,Defense_BLK_Home,Defense_PF_Home,Defense_PTS_Home
0,0,CLE,2.60,CHI,1.555556,2015,1,241.2,37.7,82.2,...,21.0,0.749,11.7,31.7,43.4,20.2,7.5,5.4,21.2,97.8
1,1,DET,3.55,ATL,1.312500,2015,0,241.8,37.1,85.8,...,20.1,0.764,11.6,32.1,43.6,23.5,7.4,4.9,19.6,97.1


In [3]:
# Using iloc to drop the first column
df = df.iloc[:, 1:]

In [4]:
df.columns

Index(['Visitor', 'Decimal_Visitor', 'Home', 'Home_Decimal', 'Season',
       'Home_Winner', 'Offense_MP_Visitor', 'Offense_FG_Visitor',
       'Offense_FGA_Visitor', 'Offense_FG%_Visitor', 'Offense_3P_Visitor',
       'Offense_3PA_Visitor', 'Offense_3P%_Visitor', 'Offense_2P_Visitor',
       'Offense_2PA_Visitor', 'Offense_2P%_Visitor', 'Offense_FT_Visitor',
       'Offense_FTA_Visitor', 'Offense_FT%_Visitor', 'Offense_ORB_Visitor',
       'Offense_DRB_Visitor', 'Offense_TRB_Visitor', 'Offense_AST_Visitor',
       'Offense_STL_Visitor', 'Offense_BLK_Visitor', 'Offense_PF_Visitor',
       'Offense_PTS_Visitor', 'Defense_Team_Visitor', 'Defense_MP_Visitor',
       'Defense_FG_Visitor', 'Defense_FGA_Visitor', 'Defense_FG%_Visitor',
       'Defense_3P_Visitor', 'Defense_3PA_Visitor', 'Defense_3P%_Visitor',
       'Defense_2P_Visitor', 'Defense_2PA_Visitor', 'Defense_2P%_Visitor',
       'Defense_FT_Visitor', 'Defense_FTA_Visitor', 'Defense_FT%_Visitor',
       'Defense_ORB_Visitor', 'Defe

In [5]:
#encoding teams
team_mapping = {
    'BOS': 0,
    'NYK': 1,
    'MIL': 2,
    'CLE': 3,
    'ORL': 4,
    'IND': 5,
    'PHI': 6,
    'MIA': 7,
    'CHI': 8,
    'ATL': 9,
    'BRK': 10,
    'TOR': 11,
    'CHO': 12,
    'WAS': 13,
    'DET': 14,
    'OKC': 15,
    'DEN': 16,
    'MIN': 17,
    'LAC': 18,
    'DAL': 19,
    'PHO': 20,
    'NOP': 21,
    'LAL': 22,
    'SAC': 23,
    'GSW': 24,
    'HOU': 25,
    'UTA': 26,
    'MEM': 27,
    'SAS': 28,
    'POR': 29
}
# Replace 'Visitor' and 'Home' columns using the mapping
df['Visitor'] = df['Visitor'].replace(team_mapping)
df['Home'] = df['Home'].replace(team_mapping)
df['Defense_Team_Visitor'] = df['Defense_Team_Visitor'].replace(team_mapping)
df['Defense_Team_Home'] = df['Defense_Team_Home'].replace(team_mapping)

# Display the updated DataFrame
df2=df


C:\Users\Joshua\AppData\Local\Temp\ipykernel_56268\1740641694.py:35: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Visitor'] = df['Visitor'].replace(team_mapping)
C:\Users\Joshua\AppData\Local\Temp\ipykernel_56268\1740641694.py:36: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Home'] = df['Home'].replace(team_mapping)
C:\Users\Joshua\AppData\Local\Temp\ipykernel_56268\1740641694.py:37: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explici

In [6]:
unique_values=df2['Home'].unique()
unique_values

array([ 8,  9, 24,  4,  0, 11, 10,  7, 14, 15,  2, 25, 27, 29, 20, 23, 22,
        5,  1, 18,  3,  6, 28, 16, 13, 21, 12, 17, 19, 26], dtype=int64)

In [7]:
# Find all columns that contain strings (object type)
string_columns = df2.select_dtypes(include=['object'])

# Display those columns
print(string_columns)

Empty DataFrame
Columns: []
Index: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, ...]

[9550 rows x 0 columns]


In [8]:
pd.set_option('display.max_columns', None) 

In [9]:
df3 = df2.rename(columns={
    'Offense_FG%_Visitor': 'Visitor_offense_FG%',
    'Offense_2P%_Visitor': 'Visitor_offense_2P%',
    'Offense_PTS_Visitor': 'Visitor_offense_PTS',
    'Offense_FG_Visitor': 'Visitor_offense_FG',
    'Offense_3P%_Visitor': 'Visitor_offense_3P%',
    'Defense_FG%_Visitor': 'Visitor_defense_FG%',
    'Visitor': 'Visitor_offense_Team',
    'Defense_FG%_Home': 'Home_defense_FG%',
    'Defense_TRB_Home': 'Home_defense_TRB',
    'Defense_2P%_Home': 'Home_defense_2P%',
    'Defense_AST_Home': 'Home_defense_AST',
    'Defense_BLK_Home': 'Home_defense_BLK',
    'Home': 'Home_offense_Team',
    'Offense_FG%_Home': 'Home_offense_FG%',
    'Offense_3P%_Home': 'Home_offense_3P%',
    'Defense_FG_Home': 'Home_defense_FG',
    'Defense_PTS_Home': 'Home_defense_PTS',
    'Decimal_Visitor': 'away_team_odds',
    'Home_Decimal': 'home_team_odds'})



In [10]:
df3

,Visitor_offense_Team,away_team_odds,Home_offense_Team,home_team_odds,Season,Home_Winner,Offense_MP_Visitor,Visitor_offense_FG,Offense_FGA_Visitor,Visitor_offense_FG%,Offense_3P_Visitor,Offense_3PA_Visitor,Visitor_offense_3P%,Offense_2P_Visitor,Offense_2PA_Visitor,Visitor_offense_2P%,Offense_FT_Visitor,Offense_FTA_Visitor,Offense_FT%_Visitor,Offense_ORB_Visitor,Offense_DRB_Visitor,Offense_TRB_Visitor,Offense_AST_Visitor,Offense_STL_Visitor,Offense_BLK_Visitor,Offense_PF_Visitor,Visitor_offense_PTS,Defense_Team_Visitor,Defense_MP_Visitor,Defense_FG_Visitor,Defense_FGA_Visitor,Visitor_defense_FG%,Defense_3P_Visitor,Defense_3PA_Visitor,Defense_3P%_Visitor,Defense_2P_Visitor,Defense_2PA_Visitor,Defense_2P%_Visitor,Defense_FT_Visitor,Defense_FTA_Visitor,Defense_FT%_Visitor,Defense_ORB_Visitor,Defense_DRB_Visitor,Defense_TRB_Visitor,Defense_AST_Visitor,Defense_STL_Visitor,Defense_BLK_Visitor,Defense_PF_Visitor,Defense_PTS_Visitor,Offense_MP_Home,Offense_FG_Home,Offense_FGA_Home,Home_offense_FG%,Offense_3P_Home,Offense_3PA_Home,Home_offense_3P%,Offense_2P_Home,Offense_2PA_Home,Offense_2P%_Home,Offense_FT_Home,Offense_FTA_Home,Offense_FT%_Home,Offense_ORB_Home,Offense_DRB_Home,Offense_TRB_Home,Offense_AST_Home,Offense_STL_Home,Offense_BLK_Home,Offense_PF_Home,Offense_PTS_Home,Defense_Team_Home,Defense_MP_Home,Home_defense_FG,Defense_FGA_Home,Home_defense_FG%,Defense_3P_Home,Defense_3PA_Home,Defense_3P%_Home,Defense_2P_Home,Defense_2PA_Home,Home_defense_2P%,Defense_FT_Home,Defense_FTA_Home,Defense_FT%_Home,Defense_ORB_Home,Defense_DRB_Home,Home_defense_TRB,Home_defense_AST,Defense_STL_Home,Home_defense_BLK,Defense_PF_Home,Home_defense_PTS
0,3,2.600000,8,1.555556,2015,1,241.2,37.7,82.2,0.458,10.1,27.5,0.367,27.6,54.7,0.504,17.7,23.6,0.751,11.1,31.9,43.0,22.1,7.4,4.1,18.4,103.1,3,241.2,38.1,83.5,0.456,7.7,22.5,0.343,30.3,61.0,0.497,14.8,19.9,0.745,10.8,30.3,41.1,24.1,7.8,4.5,20.5,98.7,242.4,36.6,82.9,0.442,7.9,22.3,0.353,28.7,60.6,0.474,19.7,25.2,0.783,11.7,34.0,45.7,21.7,6.3,5.8,18.2,100.8,8,242.4,37.7,86.7,0.435,6.6,19.6,0.335,31.2,67.1,0.465,15.7,21.0,0.749,11.7,31.7,43.4,20.2,7.5,5.4,21.2,97.8
1,14,3.550000,9,1.312500,2015,0,241.8,37.1,85.8,0.432,8.6,24.9,0.344,28.5,60.9,0.468,15.8,22.4,0.703,12.8,32.1,44.9,21.6,7.6,4.7,19.0,98.5,14,241.8,37.9,83.1,0.456,7.7,21.8,0.355,30.1,61.3,0.491,16.0,20.7,0.772,10.7,33.4,44.1,23.8,7.0,4.8,19.3,99.5,240.6,38.1,81.7,0.466,10.0,26.2,0.380,28.1,55.5,0.506,16.5,21.2,0.778,8.7,31.8,40.6,25.7,9.1,4.6,17.8,102.5,9,240.6,36.5,83.1,0.439,8.8,25.8,0.341,27.7,57.4,0.483,15.4,20.1,0.764,11.6,32.1,43.6,23.5,7.4,4.9,19.6,97.1
2,21,6.100000,24,1.140845,2015,1,241.2,37.9,82.9,0.457,7.1,19.3,0.370,30.8,63.6,0.484,16.4,21.8,0.751,11.5,32.0,43.5,22.0,6.7,6.2,18.7,99.4,21,241.2,37.9,83.1,0.456,6.5,19.6,0.335,31.4,63.5,0.494,16.2,21.4,0.759,10.6,30.9,41.5,20.7,6.8,5.8,18.7,98.6,240.6,41.6,87.0,0.478,10.8,27.0,0.398,30.8,60.0,0.514,16.0,20.8,0.768,10.4,34.3,44.7,27.4,9.3,6.0,19.9,110.0,24,240.6,37.0,86.4,0.428,7.2,21.4,0.337,29.7,65.0,0.458,18.7,24.5,0.763,11.7,32.8,44.5,21.0,8.3,3.6,18.8,99.9
3,13,1.555556,4,2.600000,2015,0,243.4,38.3,82.8,0.462,6.1,16.8,0.360,32.2,66.0,0.488,15.9,21.4,0.742,10.5,34.2,44.7,24.0,7.3,4.6,20.8,98.5,13,243.4,36.2,83.5,0.433,8.0,23.0,0.349,28.1,60.5,0.465,17.5,23.7,0.738,10.0,31.7,41.7,19.9,8.0,4.3,19.6,97.8,240.9,37.5,82.8,0.453,6.8,19.5,0.347,30.8,63.3,0.486,13.9,19.1,0.729,10.0,31.8,41.8,20.6,7.9,3.8,20.9,95.7,4,240.9,38.0,82.0,0.463,8.3,22.6,0.368,29.6,59.4,0.499,17.2,22.9,0.751,10.1,33.4,43.5,23.4,7.7,5.4,17.9,101.4
4,6,9.000000,0,1.076923,2015,1,241.5,33.7,82.6,0.408,8.4,26.3,0.320,25.3,56.3,0.449,16.1,23.8,0.676,11.9,30.9,42.9,20.5,9.6,5.9,21.7,92.0,6,241.5,37.1,82.7,0.449,8.0,22.9,0.350,29.1,59.8,0.487,18.7,24.7,0.756,11.5,34.8,46.3,23.7,9.5,5.5,20.2,101.0,242.4,38.9,87.9,0.443,8.0,24.6,0.327,30.9,63.3,0.488,15.4,20.5,0.754,11.1,32.7,43.8,24.5,8.2,3.6,21.2,101.4,0,242.4,38.1,84.7,0.450,7.4,22.0,0.336,30.7,62.7,0.490,17.6,23.3,0.755,10.9,33.8,44.7,21.9,7.1,5.3,18.8,101.2
...,...,.

In [11]:
X = df3[[
    'Visitor_offense_FG%',
    'Visitor_offense_2P%',
    'Visitor_offense_PTS',
    'Visitor_offense_FG',
    'Visitor_offense_3P%',
    'Visitor_defense_FG%',
    'Visitor_offense_Team',
    'Home_defense_FG%',
    'Home_defense_TRB',
    'Home_defense_2P%',
    'Home_defense_AST',
    'Home_defense_BLK',
    'Home_offense_Team',
    'Home_offense_FG%',
    'Home_offense_3P%',
    'Home_defense_FG',
    'Home_defense_PTS',
    'away_team_odds',
    'home_team_odds'
]]

y = df3['Home_Winner']
X.columns

Index(['Visitor_offense_FG%', 'Visitor_offense_2P%', 'Visitor_offense_PTS',
       'Visitor_offense_FG', 'Visitor_offense_3P%', 'Visitor_defense_FG%',
       'Visitor_offense_Team', 'Home_defense_FG%', 'Home_defense_TRB',
       'Home_defense_2P%', 'Home_defense_AST', 'Home_defense_BLK',
       'Home_offense_Team', 'Home_offense_FG%', 'Home_offense_3P%',
       'Home_defense_FG', 'Home_defense_PTS', 'away_team_odds',
       'home_team_odds'],
      dtype='object')

In [12]:
#scaling data
# Columns that should not be scaled
# columns_to_exclude = ['Visitor_offense_Team', 'away_team_odds', 'Home_offense_Team', 'home_team_odds']

# # Separate the data into the columns to scale and columns to exclude
# columns_to_scale = X.drop(columns=columns_to_exclude).columns  # All other columns

# # Create a scaler
# scaler = StandardScaler()

# # Scale the columns that need scaling
# df_scaled = pd.DataFrame(scaler.fit_transform(X[columns_to_scale]), columns=columns_to_scale)
# # joblib.dump(scaler, 'scaler_nba.joblib')
# # Concatenate the scaled columns back with the excluded columns
# df_final = pd.concat([X[columns_to_exclude].reset_index(drop=True), df_scaled.reset_index(drop=True)], axis=1)

# # Display the resulting DataFrame
# df_final.head()

In [13]:
# X = df_final[[
# 'Home_Decimal',  
# 'Decimal_Visitor',    
# 'Offense_FG%_Visitor', 
# 'Defense_FG%_Home',    
# 'Defense_TRB_Home',     
# 'Offense_2P%_Visitor',  
# 'Defense_DRB_Home',      
# 'Offense_FG%_Home',     
# 'Defense_2P%_Home',       
# 'Offense_PTS_Visitor',   
# 'Defense_AST_Home',       
# 'Offense_FG_Visitor',     
# 'Offense_3P%_Visitor',    
# 'Defense_FG%_Visitor',    
# 'Defense_BLK_Home',      
# 'Defense_FG_Home',
#         'Home',
#         'Visitor',
# 'Offense_3P%_Home',        
# 'Defense_PTS_Home']]


In [14]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Further split training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [15]:
#learning rate scheduler
def scheduler(epoch, lr):
    if epoch > 10:  # After the first 10 epochs
        lr = lr * tf.math.exp(-0.1)  # Reduce the learning rate
    return float(lr)  # Ensure the output is a float

In [16]:
# Define the model-building function
# Define the model-building function
def build_model(trial):
    model = models.Sequential()
    
    # Tune number of layers
    num_layers = trial.suggest_int('num_layers', 2, 7)  # Correctly specify low and high

    for i in range(num_layers):
        # Specify the number of units with the correct method
        units = trial.suggest_int(f'units_{i}', 16, 1056)  # Specify name and bounds
        
        # Suggest activation functions
        activation = trial.suggest_categorical('activation', ['relu', 'tanh', 'sigmoid'])  # Correct method
        
        # Use suggest_float for dropout rate
        dropout_rate = trial.suggest_float(f'dropout_{i}', 0.1, 0.5)  # Correct method for float
        
        l2_reg = trial.suggest_float('l2', 1e-7, 1e-3, log=True)  # Replace with suggest_float

        
        model.add(layers.Dense(
            units=units,
            activation=activation,
            kernel_initializer=trial.suggest_categorical('kernel_initializer', ['glorot_uniform']),  # Correct method
            kernel_regularizer=regularizers.l2(l2_reg)  # Correctly use regularizers

        ))

        use_batch_norm = trial.suggest_categorical('use_batch_norm', [True, False])
        if use_batch_norm:
            model.add(layers.BatchNormalization())

        model.add(layers.Dropout(dropout_rate))
    
    model.add(layers.Dense(1, activation='sigmoid'))
    
    # Choose optimizer
    optimizer_name = trial.suggest_categorical('optimizer', ['adam', 'sgd', 'rmsprop'])  # Correct method
    if optimizer_name == 'adam':
        optimizer = optimizers.Adam(
            learning_rate=trial.suggest_float('learning_rate', 1e-5, 1e-1),
            beta_1=trial.suggest_float('beta_1', 0.5, 0.99),
            beta_2=trial.suggest_float('beta_2', 0.9, 0.999),
            epsilon=trial.suggest_float('epsilon', 1e-8, 1e-6)
        )
    elif optimizer_name == 'sgd':
        optimizer = optimizers.SGD(
            learning_rate=trial.suggest_float('learning_rate', 1e-5, 1e-1),
            momentum=trial.suggest_float('momentum', 0.0, 0.9)  # Correctly suggest momentum
        )
    else:
        optimizer = optimizers.RMSprop(
            learning_rate=trial.suggest_float('learning_rate', 1e-5, 1e-1)
        )
    
    model.compile(
        optimizer=optimizer,
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    return model



# Objective function for Optuna
# Objective function for Optuna
def objective(trial):

    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    validation_scores = []

    for train_index, val_index in kf.split(X):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]

    
    model = build_model(trial)
    
    # Implement early stopping
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True
    )
    # Learning Rate Scheduler
    lr_scheduler = callbacks.LearningRateScheduler(scheduler)

    
    # Implement model checkpoint
    checkpoint_callback = ModelCheckpoint(
        filepath=r'C:\Users\Joshua\desktop\nba_third_try\model\not_scaled_nba.keras',
        monitor='val_loss',
        save_best_only=True
    )
    
    # Train the model
    history = model.fit(
        X_train, y_train,
        epochs=trial.suggest_int('epochs', 10, 100),  # Use suggest_int for epochs
        batch_size=trial.suggest_categorical('batch_size', [16, 32, 64, 128]),  # Use suggest_categorical for batch size
        validation_data=(X_val, y_val),
        callbacks=[early_stopping, checkpoint_callback, lr_scheduler],
        verbose=0
    )
    
    val_accuracy = history.history['val_accuracy'][-1]
    return val_accuracy


# Define Optuna study
sampler = TPESampler(n_startup_trials=8)
study = optuna.create_study(sampler=sampler, direction='maximize')
study.optimize(objective, n_trials=100)  # Adjust n_trials based on computational resources

# Print the best hyperparameters
print("Best hyperparameters found:")
print(study.best_params)

# Load the best model
best_model = models.load_model(r'C:\Users\Joshua\desktop\nba_third_try\model\not_scaled_nba.keras')

[I 2024-11-06 15:18:18,897] A new study created in memory with name: no-name-3efa7812-6a7c-47d3-8499-58bbe518604b
[I 2024-11-06 15:19:07,161] Trial 0 finished with value: 0.6638743281364441 and parameters: {'num_layers': 5, 'units_0': 934, 'activation': 'relu', 'dropout_0': 0.3843528901244361, 'l2': 0.00033955494693743596, 'kernel_initializer': 'glorot_uniform', 'use_batch_norm': True, 'units_1': 451, 'dropout_1': 0.14058955713510835, 'units_2': 428, 'dropout_2': 0.42162295257049287, 'units_3': 488, 'dropout_3': 0.20093962985743033, 'units_4': 204, 'dropout_4': 0.18861907764894786, 'optimizer': 'adam', 'learning_rate': 0.09584776948168051, 'beta_1': 0.9807678062276763, 'beta_2': 0.9965356747947451, 'epsilon': 8.40761736205325e-08, 'epochs': 75, 'batch_size': 64}. Best is trial 0 with value: 0.6638743281364441.
[I 2024-11-06 15:19:15,846] Trial 1 finished with value: 0.563874363899231 and parameters: {'num_layers': 5, 'units_0': 418, 'activation': 'tanh', 'dropout_0': 0.4366521798177132

Best hyperparameters found:
{'num_layers': 3, 'units_0': 245, 'activation': 'sigmoid', 'dropout_0': 0.15542480162415667, 'l2': 1.5056663080210335e-05, 'kernel_initializer': 'glorot_uniform', 'use_batch_norm': False, 'units_1': 937, 'dropout_1': 0.1831259744641057, 'units_2': 986, 'dropout_2': 0.14756994005540486, 'optimizer': 'rmsprop', 'learning_rate': 0.0012711550817668844, 'epochs': 100, 'batch_size': 128}


In [17]:
# V = df[['Visitor', 'Decimal_Visitor', 'Home', 'Home_Decimal', 'Season',
#     'Offense_MP_Visitor', 'Offense_FG_Visitor',
#        'Offense_FGA_Visitor', 'Offense_FG%_Visitor', 'Offense_3P_Visitor',
#        'Offense_3PA_Visitor', 'Offense_3P%_Visitor', 'Offense_2P_Visitor',
#        'Offense_2PA_Visitor', 'Offense_2P%_Visitor', 'Offense_FT_Visitor',
#        'Offense_FTA_Visitor', 'Offense_FT%_Visitor', 'Offense_ORB_Visitor',
#        'Offense_DRB_Visitor', 'Offense_TRB_Visitor', 'Offense_AST_Visitor',
#        'Offense_STL_Visitor', 'Offense_BLK_Visitor', 'Offense_PF_Visitor',
#        'Offense_PTS_Visitor', 'Defense_Team_Visitor', 'Defense_MP_Visitor',
#        'Defense_FG_Visitor', 'Defense_FGA_Visitor', 'Defense_FG%_Visitor',
#        'Defense_3P_Visitor', 'Defense_3PA_Visitor', 'Defense_3P%_Visitor',
#        'Defense_2P_Visitor', 'Defense_2PA_Visitor', 'Defense_2P%_Visitor',
#        'Defense_FT_Visitor', 'Defense_FTA_Visitor', 'Defense_FT%_Visitor',
#        'Defense_ORB_Visitor', 'Defense_DRB_Visitor', 'Defense_TRB_Visitor',
#        'Defense_AST_Visitor', 'Defense_STL_Visitor', 'Defense_BLK_Visitor',
#        'Defense_PF_Visitor', 'Defense_PTS_Visitor', 'Offense_MP_Home',
#        'Offense_FG_Home', 'Offense_FGA_Home', 'Offense_FG%_Home',
#        'Offense_3P_Home', 'Offense_3PA_Home', 'Offense_3P%_Home',
#        'Offense_2P_Home', 'Offense_2PA_Home', 'Offense_2P%_Home',
#        'Offense_FT_Home', 'Offense_FTA_Home', 'Offense_FT%_Home',
#        'Offense_ORB_Home', 'Offense_DRB_Home', 'Offense_TRB_Home',
#        'Offense_AST_Home', 'Offense_STL_Home', 'Offense_BLK_Home',
#        'Offense_PF_Home', 'Offense_PTS_Home', 'Defense_Team_Home',
#        'Defense_MP_Home', 'Defense_FG_Home', 'Defense_FGA_Home',
#        'Defense_FG%_Home', 'Defense_3P_Home', 'Defense_3PA_Home',
#        'Defense_3P%_Home', 'Defense_2P_Home', 'Defense_2PA_Home',
#        'Defense_2P%_Home', 'Defense_FT_Home', 'Defense_FTA_Home',
#        'Defense_FT%_Home', 'Defense_ORB_Home', 'Defense_DRB_Home',
#        'Defense_TRB_Home', 'Defense_AST_Home', 'Defense_STL_Home',
#        'Defense_BLK_Home', 'Defense_PF_Home', 'Defense_PTS_Home', 'Home_Winner']]


In [18]:
# # Set display options to show all rows and avoid truncation
# pd.set_option('display.max_rows', None)

# # Calculate the correlation matrix (assuming 'df' contains the data)
# correlation_matrix = V.corr()

# # Print the full list of correlations with the target variable, sorted by absolute value
# correlation_with_target = correlation_matrix['Home_Winner'].sort_values(key=abs, ascending=False)
# print(correlation_with_target)

# # Visualize the correlation matrix with a heatmap
# plt.figure(figsize=(10, 8))
# sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap='coolwarm')
# plt.title('Correlation Matrix')
# plt.show()

# # Reset display options after printing
# pd.reset_option('display.max_rows')
